In [1]:
import pandas as pd
import os

In [2]:
mcsa_aug_dataset_path = '../../dataset/mcsa_fine_tune/mcsa_aug_20_mutation_rate_0.2_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123'

train_valid_fasta_name = 'train_valid_aug_sequence.fasta'
train_valid_fasta_path = os.path.join(mcsa_aug_dataset_path, train_valid_fasta_name)

rfdiffusion_case_fasta_path = '../../dataset/rfdiffusion_enzyme_design/Active_site_scaffolds_reformate.fasta'

In [3]:
def read_blast_results(path):
    column_headers = [
    "Query ID",
    "Subject ID",
    "% Identity",
    "Alignment Length",
    "Mismatches",
    "Gap Opens",
    "Query Start",
    "Query End",
    "Subject Start",
    "Subject End",
    "E-value",
    "Bit Score",
    ]
    try:
        results_df = pd.read_csv(path, sep='\t', header=None)
        results_df.columns = column_headers

        return results_df
    except:
        print('没有Blast的结果')
        return pd.DataFrame()


In [4]:
import subprocess

database_fasta = train_valid_fasta_path
database = os.path.join(os.path.dirname(database_fasta), 'train_valid_aug_sequence')
command = f'makeblastdb -in {database_fasta} -dbtype prot -out {database}'
subprocess.run(command, shell=True)



Building a new DB, current time: 07/12/2024 13:31:24
New DB name:   /home/xiaoruiwang/data/ubuntu_work_beta/single_step_work/EasIFA_v2/dataset/mcsa_fine_tune/mcsa_aug_20_mutation_rate_0.2_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123/train_valid_aug_sequence
New DB title:  ../../dataset/mcsa_fine_tune/mcsa_aug_20_mutation_rate_0.2_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123/train_valid_aug_sequence.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /home/xiaoruiwang/data/ubuntu_work_beta/single_step_work/EasIFA_v2/dataset/mcsa_fine_tune/mcsa_aug_20_mutation_rate_0.2_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123/train_valid_aug_sequence
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 17420 sequences in 0.156296 seconds.


CompletedProcess(args='makeblastdb -in ../../dataset/mcsa_fine_tune/mcsa_aug_20_mutation_rate_0.2_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123/train_valid_aug_sequence.fasta -dbtype prot -out ../../dataset/mcsa_fine_tune/mcsa_aug_20_mutation_rate_0.2_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123/train_valid_aug_sequence', returncode=0)

In [5]:
query_file = rfdiffusion_case_fasta_path
output_file = os.path.join('../rfdiffusion_case_study/data', 'blast_results_use_mcsa_aug.txt')
command = f'blastp -query {query_file} -db {database} -out {output_file} -evalue 0.001 -outfmt 6 -num_threads 16'    # 这里的evalue=0.001是一个较为宽松的阈值

subprocess.run(command, shell=True)
# subprocess.run(command, shell=True)

CompletedProcess(args='blastp -query ../../dataset/rfdiffusion_enzyme_design/Active_site_scaffolds_reformate.fasta -db ../../dataset/mcsa_fine_tune/mcsa_aug_20_mutation_rate_0.2_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123/train_valid_aug_sequence -out ../rfdiffusion_case_study/data/blast_results_use_mcsa_aug.txt -evalue 0.001 -outfmt 6 -num_threads 16', returncode=0)

In [6]:
blast_p_results = read_blast_results(path=output_file)
blast_p_results

没有Blast的结果


""


In [7]:
if not blast_p_results.empty:
    blast_p_results.sort_values('% Identity', ascending=False)

In [8]:
# 合并qurey fasta和database fasta，使用CD-HIT进行聚类，获取相似度等级
! cat ../../dataset/mcsa_fine_tune/mcsa_aug_20_mutation_rate_0.2_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123/train_valid_aug_sequence.fasta > ../rfdiffusion_case_study/data/rfdiffusion_train_valid_test_merge.fasta
! cat ../../dataset/rfdiffusion_enzyme_design/Active_site_scaffolds_reformate.fasta >> ../rfdiffusion_case_study/data/rfdiffusion_train_valid_test_merge.fasta

merge_fasta_path = '../rfdiffusion_case_study/data/rfdiffusion_train_valid_test_merge.fasta'

In [9]:
# 使用CD-HIT聚类，查看相似度

# 定义CD-HIT命令和参数

def run_subprocess_and_print(args):
    # 开启一个新的进程
    with subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True) as proc:
        # 实时读取输出
        while True:
            output = proc.stdout.readline()
            if output == '' and proc.poll() is not None:
                break
            if output:
                print(output.strip())  # 输出每一行
        # 检查错误输出
        err = proc.stderr.read()
        if err:
            print("Error:", err)
    proc.communicate()

threshold = [0.4]
command = "/home/xiaoruiwang/software/cdhit/cd-hit"
for thre in threshold:

    if thre < 0.5:
        word = 2
    elif thre < 0.6:
        word = 3
    elif thre < 0.7:
        word = 4
    else:
        word = 5
    args = ["-i", os.path.abspath(merge_fasta_path),
            "-o", os.path.abspath(os.path.join(os.path.dirname(merge_fasta_path), f'rfdiffusion_train_valid_test_merge_sequence_cutoff_{str(thre)}.fasta')),
            "-c", str(thre),
            "-n", str(word),
            "-T", str(12),
            "-d", '0']
    print('Args: {}'.format(' '.join(args)))
    if not os.path.exists(os.path.abspath(os.path.join(os.path.dirname(merge_fasta_path), f'rfdiffusion_train_valid_test_merge_sequence_cutoff_{str(thre)}.fasta'))):
        run_subprocess_and_print([command] + args)


Args: -i /home/xiaoruiwang/data/ubuntu_work_beta/single_step_work/EasIFA_v2/script/rfdiffusion_case_study/data/rfdiffusion_train_valid_test_merge.fasta -o /home/xiaoruiwang/data/ubuntu_work_beta/single_step_work/EasIFA_v2/script/rfdiffusion_case_study/data/rfdiffusion_train_valid_test_merge_sequence_cutoff_0.4.fasta -c 0.4 -n 2 -T 12 -d 0


In [10]:
fasta_dir_path = os.path.abspath(os.path.dirname(merge_fasta_path))

In [11]:
from collections import defaultdict
def get_cluster(cluster_results_path):
    results = defaultdict(list)
    with open(cluster_results_path, 'r', encoding='utf-8') as f:
        data = [x.strip() for x in f.readlines()]
    for line in data:
        if '>Cluster' in line:
            cluster = line.replace('>', '')
            continue
        sequence_id = line.split('>')[-1].split('.')[0]
        results[cluster].append(sequence_id)
    return results


def id2cluster(cluster_results:dict):
    sequence_id2cluster = {}
    for key in cluster_results:
        for id in cluster_results[key]:
            sequence_id2cluster[id] = key
    return sequence_id2cluster


In [12]:
threshold = threshold
sequence_id2cluster_all_levels = {}
cluster_all_levels2sequence_id = {}
for thre in threshold:
    cluster_results_path = os.path.abspath(os.path.join(fasta_dir_path, f'rfdiffusion_train_valid_test_merge_sequence_cutoff_{str(thre)}.fasta.clstr'))
    cluster_results = get_cluster(cluster_results_path)
    print(len(cluster_results))
    sequence_id2cluster_all_levels[thre] = id2cluster(cluster_results)
    cluster_all_levels2sequence_id[thre] = cluster_results

1016


In [13]:
sequence_id2cluster_all_levels[0.4]['EC2_AF2_2']

'Cluster 430'

In [14]:
cluster_all_levels2sequence_id[0.4][sequence_id2cluster_all_levels[0.4]['EC2_AF2_2']] 

['EC2_AF2_2']

In [15]:
cluster_all_levels2sequence_id[0.4][sequence_id2cluster_all_levels[0.4]['EC4_AF2_28']]

['EC4_AF2_28']

In [16]:
cluster_all_levels2sequence_id[0.4][sequence_id2cluster_all_levels[0.4]['EC3_AF2_3']]    # 与生成的增强数据P81054-c0_m17 有 40.54% 相似度（见script/rfdiffusion_case_study/data/rfdiffusion_train_valid_test_merge_sequence_cutoff_0.4.fasta.clstr）

['P81054-c0_m17', 'EC3_AF2_3']

In [17]:
cluster_all_levels2sequence_id[0.4][sequence_id2cluster_all_levels[0.4]['EC5_AF2_6']]

['EC5_AF2_6']

In [18]:
! cat ../../dataset/ec_site_dataset/uniprot_ecreact_cluster_split_merge_dataset_limit_100/blast_database.fasta >> ../rfdiffusion_case_study/data/swissprot_erxn_asa_rfdiffusion_train_valid_test_merge.fasta
! cat ../rfdiffusion_case_study/data/rfdiffusion_train_valid_test_merge.fasta >> ../rfdiffusion_case_study/data/swissprot_erxn_asa_rfdiffusion_train_valid_test_merge.fasta

all_merge_fasta_path = '../rfdiffusion_case_study/data/swissprot_erxn_asa_rfdiffusion_train_valid_test_merge.fasta'

In [19]:
# 使用CD-HIT聚类，查看相似度

# 定义CD-HIT命令和参数

def run_subprocess_and_print(args):
    # 开启一个新的进程
    with subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True) as proc:
        # 实时读取输出
        while True:
            output = proc.stdout.readline()
            if output == '' and proc.poll() is not None:
                break
            if output:
                print(output.strip())  # 输出每一行
        # 检查错误输出
        err = proc.stderr.read()
        if err:
            print("Error:", err)
    proc.communicate()

threshold = [0.4]
command = "/home/xiaoruiwang/software/cdhit/cd-hit"
for thre in threshold:

    if thre < 0.5:
        word = 2
    elif thre < 0.6:
        word = 3
    elif thre < 0.7:
        word = 4
    else:
        word = 5
    args = ["-i", os.path.abspath(all_merge_fasta_path),
            "-o", os.path.abspath(os.path.join(os.path.dirname(all_merge_fasta_path), f'swissprot_erxn_asa_rfdiffusion_train_valid_test_merge_sequence_cutoff_{str(thre)}.fasta')),
            "-c", str(thre),
            "-n", str(word),
            "-T", str(12),
            "-d", '0']
    print('Args: {}'.format(' '.join(args)))
    if not os.path.exists(os.path.abspath(os.path.join(os.path.dirname(all_merge_fasta_path), f'swissprot_erxn_asa_rfdiffusion_train_valid_test_merge_sequence_cutoff_{str(thre)}.fasta'))):
        run_subprocess_and_print([command] + args)


Args: -i /home/xiaoruiwang/data/ubuntu_work_beta/single_step_work/EasIFA_v2/script/rfdiffusion_case_study/data/swissprot_erxn_asa_rfdiffusion_train_valid_test_merge.fasta -o /home/xiaoruiwang/data/ubuntu_work_beta/single_step_work/EasIFA_v2/script/rfdiffusion_case_study/data/swissprot_erxn_asa_rfdiffusion_train_valid_test_merge_sequence_cutoff_0.4.fasta -c 0.4 -n 2 -T 12 -d 0


In [20]:
threshold = threshold
sequence_id2cluster_all_levels_add_swissprot_erxn_asa = {}
cluster_all_levels2sequence_id_add_swissprot_erxn_asa = {}
for thre in threshold:
    cluster_results_path_add_swissprot_erxn_asa = os.path.abspath(os.path.join(fasta_dir_path, f'swissprot_erxn_asa_rfdiffusion_train_valid_test_merge_sequence_cutoff_{str(thre)}.fasta.clstr'))
    cluster_results_add_swissprot_erxn_asa = get_cluster(cluster_results_path_add_swissprot_erxn_asa)
    print(len(cluster_results_add_swissprot_erxn_asa))
    sequence_id2cluster_all_levels_add_swissprot_erxn_asa[thre] = id2cluster(cluster_results_add_swissprot_erxn_asa)
    cluster_all_levels2sequence_id_add_swissprot_erxn_asa[thre] = cluster_results_add_swissprot_erxn_asa

8147


In [21]:
sequence_id2cluster_all_levels_add_swissprot_erxn_asa[0.4]['EC2_AF2_2']

'Cluster 7560'

In [22]:
cluster_all_levels2sequence_id_add_swissprot_erxn_asa[0.4][sequence_id2cluster_all_levels_add_swissprot_erxn_asa[0.4]['EC2_AF2_2']] 

['EC2_AF2_2']

In [23]:
cluster_all_levels2sequence_id_add_swissprot_erxn_asa[0.4][sequence_id2cluster_all_levels_add_swissprot_erxn_asa[0.4]['EC4_AF2_28']] 

['EC4_AF2_28']

In [24]:
cluster_all_levels2sequence_id_add_swissprot_erxn_asa[0.4][sequence_id2cluster_all_levels_add_swissprot_erxn_asa[0.4]['EC3_AF2_3']] 

['EC3_AF2_3']

In [25]:
cluster_all_levels2sequence_id_add_swissprot_erxn_asa[0.4][sequence_id2cluster_all_levels_add_swissprot_erxn_asa[0.4]['EC5_AF2_6']] 

['EC5_AF2_6']

In [26]:
not_unique_claster = []
for ec, file_flag in zip(range(1,6), range(0, 100)):
    try:
        cluster_all_levels2sequence_id_add_swissprot_erxn_asa[0.4][sequence_id2cluster_all_levels_add_swissprot_erxn_asa[0.4][f'EC{ec}_AF2_{file_flag}']]
        if len(cluster_all_levels2sequence_id_add_swissprot_erxn_asa[0.4][sequence_id2cluster_all_levels_add_swissprot_erxn_asa[0.4][f'EC{ec}_AF2_{file_flag}']]) != 1:
            not_unique_claster.append(cluster_all_levels2sequence_id_add_swissprot_erxn_asa[0.4][sequence_id2cluster_all_levels_add_swissprot_erxn_asa[0.4][f'EC{ec}_AF2_{file_flag}']])
    except:
        pass

if len(not_unique_claster) == 0:
    print('All the artificial enzymes formed a single cluster.')
else:
    print(not_unique_claster)


All the artificial enzymes formed a single cluster.
